In [2]:
!wget -nc http://images.cocodataset.org/zips/val2017.zip
!unzip -q -n val2017.zip -d {dataset_dir}

!cd {dataset_dir}

# Download annotations
!wget -nc http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -q -n annotations_trainval2017.zip -d {dataset_dir}
!cp {os.path.join(dataset_dir,'annotations','instances_val2017.json')} {os.path.join(dataset_dir,'val2017')}

--2025-01-16 10:57:43--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.130.193, 52.217.116.177, 52.217.118.9, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.130.193|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777.80M  55.7MB/s    in 15s     

2025-01-16 10:57:59 (51.6 MB/s) - ‘val2017.zip’ saved [815585330/815585330]

--2025-01-16 10:58:05--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.142.188, 52.217.195.33, 52.217.138.153, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.142.188|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import engine as SE
import transforms as ST
import utils as SU
from torchvision.datasets import CocoDetection
import torchvision.models.detection as M

In [4]:
# Define data transforms
transforms = ST.Compose([ST.ToTensor()])

# Create dataset
dataset = CocoDetection('/content/{dataset_dir}/val2017', '/content/{dataset_dir}/annotations/instances_val2017.json', transforms = transforms)

# Create data loader
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2, collate_fn=SU.collate_fn)

loading annotations into memory...
Done (t=0.69s)
creating index...
index created!


In [ ]:
# Load the pre-trained Faster R-CNN model
print('Loading pre-trained model...')

model_name = 'Faster R-CNN ResNet-50 FPN' #@param ['Faster R-CNN ResNet-50 FPN', 'RetinaNet ResNet-50 FPN', 'SSD300 VGG16']

if model_name == 'Faster R-CNN ResNet-50 FPN':
  mode_id = 'fasterrcnn_resnet50_fpn'
  model = fasterrcnn_resnet50_fpn(pretrained=True, force_reload=True)
  image_res = (3,800,800)

elif model_name == 'RetinaNet ResNet-50 FPN':
  mode_id = 'retinanet_resnet50_fpn'
  model = retinanet_resnet50_fpn(pretrained=True)
  image_res = (3,800,800)

elif model_name == 'SSD300 VGG16':
  mode_id = 'ssd300_vgg16'
  model = ssd300_vgg16(pretrained=True)
  image_res = (3,300,300)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSD300_VGG16_Weights.COCO_V1`. You can also use `weights=SSD300_VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/ssd300_vgg16_coco-b556d3b4.pth" to /root/.cache/torch/hub/checkpoints/ssd300_vgg16_coco-b556d3b4.pth
100%|██████████| 136M/136M [00:01<00:00, 98.4MB/s]


In [ ]:
# Clear cache for accurate maximum GPU memory usage logging
torch.cuda.empty_cache()

# Get appropriate device for model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Evaluate model
evaluator, fps, max_mem, outputs = SE.evaluate(model, data_loader, device, img_size=image_res[1])

Test:  [   0/5000]  eta: 3:01:06  model_time: 1.9809 (1.9809)  evaluator_time: 0.0279 (0.0279)  time: 2.1734  data: 0.1618  max mem: 208
Test:  [ 100/5000]  eta: 0:07:00  model_time: 0.0490 (0.0615)  evaluator_time: 0.0182 (0.0174)  time: 0.0787  data: 0.0051  max mem: 210
Test:  [ 200/5000]  eta: 0:05:52  model_time: 0.0484 (0.0505)  evaluator_time: 0.0153 (0.0171)  time: 0.0812  data: 0.0055  max mem: 210
Test:  [ 300/5000]  eta: 0:05:22  model_time: 0.0464 (0.0464)  evaluator_time: 0.0179 (0.0170)  time: 0.0833  data: 0.0069  max mem: 210
Test:  [ 400/5000]  eta: 0:04:57  model_time: 0.0329 (0.0439)  evaluator_time: 0.0104 (0.0162)  time: 0.0463  data: 0.0022  max mem: 210
Test:  [ 500/5000]  eta: 0:04:37  model_time: 0.0334 (0.0419)  evaluator_time: 0.0093 (0.0155)  time: 0.0507  data: 0.0021  max mem: 210
Test:  [ 600/5000]  eta: 0:04:34  model_time: 0.0337 (0.0414)  evaluator_time: 0.0108 (0.0167)  time: 0.0502  data: 0.0024  max mem: 210
Test:  [ 700/5000]  eta: 0:04:22  model_t

In [ ]:
!pip install ptflops

In [ ]:
from ptflops import get_model_complexity_info

macs, params = get_model_complexity_info(model, image_res)
macs = round(int(macs) / 1000000000, 1)
params = round(int(params) / 1000000, 1)

print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

AssertionError: 

In [ ]:
model_id = 'ssd300_vgg16'

cols = ["model", "model_img_size_arr", "iou_type", "metric", "iou", "area", "max_dets", "score", "gpu"]
model = []
model_img_size_arr = []
model_params_arr = []
model_gflops_arr = []
iou_type = []
metric = []
iou = []
area = []
max_dets = []
score = []
gpu = []

for curr_iou_type, coco_eval in evaluator.coco_eval.items():

  model += [model_id for i in range(0,12)]
  model_img_size_arr += [image_res[1] for i in range(0,12)]
  iou_type += [curr_iou_type for i in range(0,12)]
  metric += ['avg_precision' for i in range(0,6)] + ['avg_recall' for i in range(0,6)]
  iou += ['0.50:0.95', '0.50', '0.75'] + ['0.50:0.95' for i in range(0,9)]
  area += ['all' for i in range(0,3)] + ['small', 'medium', 'large'] + ['all' for i in range(0,3)] + ['small', 'medium', 'large']
  max_dets += [100 for i in range(0,6)] + [1, 10] + [100 for i in range(0,4)]
  #model_params_arr += [params for i in range(0,12)]
  #model_gflops_arr += [macs for i in range(0,12)]
  score += list(coco_eval.stats)
  gpu += [torch.cuda.get_device_name(0) for i in range(0,12)]

In [ ]:
import pandas as pd
import numpy as np
import json

results = pd.DataFrame(np.column_stack([model, model_img_size_arr, iou_type, metric, iou, area, max_dets, score, gpu]))
print(results)
results.columns = cols

               0    1     2              3          4       5    6  \
0   ssd300_vgg16  800  bbox  avg_precision  0.50:0.95     all  100   
1   ssd300_vgg16  800  bbox  avg_precision       0.50     all  100   
2   ssd300_vgg16  800  bbox  avg_precision       0.75     all  100   
3   ssd300_vgg16  800  bbox  avg_precision  0.50:0.95   small  100   
4   ssd300_vgg16  800  bbox  avg_precision  0.50:0.95  medium  100   
5   ssd300_vgg16  800  bbox  avg_precision  0.50:0.95   large  100   
6   ssd300_vgg16  800  bbox     avg_recall  0.50:0.95     all    1   
7   ssd300_vgg16  800  bbox     avg_recall  0.50:0.95     all   10   
8   ssd300_vgg16  800  bbox     avg_recall  0.50:0.95     all  100   
9   ssd300_vgg16  800  bbox     avg_recall  0.50:0.95   small  100   
10  ssd300_vgg16  800  bbox     avg_recall  0.50:0.95  medium  100   
11  ssd300_vgg16  800  bbox     avg_recall  0.50:0.95   large  100   

                      7         8  
0    0.2510498390266604  Tesla T4  
1    0.4156977032